## Expiriments with neural-editor

I'm sure there is an easy fix but to get this to work I needed to make two symlinks in the 

In [1]:


### YOU NEED TO CHANGE THIS TO YOUR PATH
DATA_DIR = "/scratch/jl4722/neural-editor-slurm/neural-editor/" 


Set the $TEXTMORPH_DATA environment variable

In [2]:
import os 
env_var = "TEXTMORPH_DATA"


os.environ[env_var] = "{}{}".format(DATA_DIR, "data")

Make sure nltk is installed (this is a third party tool they use to tokenize the input)

In [3]:
%%bash
python -m nltk.downloader punkt

[nltk_data] Downloading package punkt to /home/jl4722/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


import packages

In [4]:
import argparse

from gtd.io import save_stdout
from gtd.log import set_log_level
from gtd.utils import Config
from textmorph.edit_model.training_run import EditTrainingRuns

set_log_level('DEBUG')

setup arguments for Experiment

In [5]:
arg_parser = argparse.ArgumentParser()
arg_parser.add_argument('exp_id', nargs='+')
arg_parser.add_argument('-c', '--check_commit', default='strict')
arg_parser.add_argument('-p', '--profile', action='store_true')

args = arg_parser.parse_args(['1'])

In [6]:
from textmorph import data

In [7]:
# experiments = EditTrainingRuns(check_commit=(args.check_commit=='strict'))
experiments = EditTrainingRuns(check_commit=(False))

In [8]:
RUN_NUM = 1

exp = experiments.get(1)

Reloaded checkpoint #350000



[localhost] local: wc -l /scratch/jl4722/neural-editor-slurm/neural-editor/data/onebillion_split/train.tsv



[localhost] local: wc -l /scratch/jl4722/neural-editor-slurm/neural-editor/data/onebillion_split/valid.tsv



[localhost] local: wc -l /scratch/jl4722/neural-editor-slurm/neural-editor/data/onebillion_split/test.tsv


INFO:root:Reloaded TrainingRun #1


In [9]:
editor = exp.editor

In [10]:
from textmorph.edit_model.editor import EditExample

ex = EditExample.from_prompt()

Enter a source sentence:
The breach shocked analysts
Enter whitelist (OK to leave empty):
catastrophic
Enter blacklist (OK to leave empty):

Enter target sentence (OK to leave empty):



In [11]:
print(ex)

SOURCE: The breach shocked analysts
INSERT EXACT: [u'catastrophic']


In [12]:
output_words_batch, edit_traces = editor.edit([ex], beam_size=8)

In [13]:
print(output_words_batch)

[[[u'the', u'lack', u'of', u'security', u'guards', u'shocked', u'people', u'shocked', u'investors', u'shocked', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u'investors', u'shocked', u'investors', u'shocked', u'.'], [u'the', u'lack', u'of', u'security', u'guards', u'shocked', u'people', u'shocked', u'investors', u'shocked', u'investors', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u'investors', u'shocked', u'to', u'bst', u'<date>', u'.'], [u'the', u'lack', u'of', u'security', u'guards', u'shocked', u'people', u'shocked', u'investors', u'shocked', u'investors', u'shocked', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u'investors', u'shocked', u'investors', u'shocked', u'investors', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u',', u'shocked', u'shocked', u'investors', u'shocked', u'investors', u'shocked'

In [14]:
print(edit_traces)

[SOURCE: The breach shocked analysts
INSERT EXACT: [u'catastrophic']
(0.00) the lack of security guards shocked people shocked investors shocked .
(0.00) the lack of security doesn ' m shocked shocked investors shocked investors shocked .
(0.00) the lack of security guards shocked people shocked investors shocked investors .
(0.00) the lack of security doesn ' m shocked shocked investors shocked to bst <date> .
(0.00) the lack of security guards shocked people shocked investors shocked investors shocked .]


In [15]:
output_words = output_words_batch[0]
edit_trace = edit_traces[0]

In [16]:
print(output_words)

[[u'the', u'lack', u'of', u'security', u'guards', u'shocked', u'people', u'shocked', u'investors', u'shocked', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u'investors', u'shocked', u'investors', u'shocked', u'.'], [u'the', u'lack', u'of', u'security', u'guards', u'shocked', u'people', u'shocked', u'investors', u'shocked', u'investors', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u'investors', u'shocked', u'to', u'bst', u'<date>', u'.'], [u'the', u'lack', u'of', u'security', u'guards', u'shocked', u'people', u'shocked', u'investors', u'shocked', u'investors', u'shocked', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u'investors', u'shocked', u'investors', u'shocked', u'investors', u'.'], [u'the', u'lack', u'of', u'security', u'doesn', u"'", u'm', u'shocked', u'shocked', u',', u'shocked', u'shocked', u'investors', u'shocked', u'investors', u'shocked',

In [17]:
edit_trace

SOURCE: The breach shocked analysts
INSERT EXACT: [u'catastrophic']
(0.00) the lack of security guards shocked people shocked investors shocked .
(0.00) the lack of security doesn ' m shocked shocked investors shocked investors shocked .
(0.00) the lack of security guards shocked people shocked investors shocked investors .
(0.00) the lack of security doesn ' m shocked shocked investors shocked to bst <date> .
(0.00) the lack of security guards shocked people shocked investors shocked investors shocked .

In [18]:
from nltk import word_tokenize
from textmorph.edit_model.editor import EditExample

In [19]:
input_sentance = "A spokesman for the organization unveiled the plan last week."
insert_words = "spokesperson"
delete_words = "spokesman"
target_sentance = "A spokesperson for the organization unveiled the plan last week."


source_words  = word_tokenize(input_sentance)
whitelist     = sorted(word_tokenize(insert_words))
blacklist     = sorted(word_tokenize(delete_words)) 
target_words  = word_tokenize(target_sentance)



In [20]:
ex = EditExample(source_words, [], whitelist, [], blacklist, target_words)

output_words_batch, edit_traces = editor.edit([ex], beam_size=8)
print(edit_traces)


[SOURCE: A spokesman for the organization unveiled the plan last week .
INSERT EXACT: ['spokesperson']
DELETE EXACT: ['spokesman']
TARGET: A spokesperson for the organization unveiled the plan last week .
(0.00) a spokesman for <org> said the group was reviewing the decision .
(0.00) a spokesman for <org> , <person> , said the group was reviewing the decision .
(0.00) a spokesman for <org> , <person> , said the soldier was healthy .
(0.00) a spokesman for <org> , <org> , said the agency was reviewing the decision .
(0.00) a spokesman for <org> , <org> , said the group was reviewing the decision .]


### get the internal states

In [21]:
ex.edit_embed

In [22]:
ex

SOURCE: A spokesman for the organization unveiled the plan last week .
INSERT EXACT: ['spokesperson']
DELETE EXACT: ['spokesman']
TARGET: A spokesperson for the organization unveiled the plan last week .

In [23]:
dir(ex.edit_embed)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [24]:
editor.encoder.source_encoder

MultiLayerSourceEncoder (
  (encoder_layer_0): BidirectionalSourceEncoder (
    (forward_encoder): SimpleSourceEncoder (
      (rnn_cell): LSTMCell(300, 128)
    )
    (backward_encoder): SimpleSourceEncoder (
      (rnn_cell): LSTMCell(300, 128)
    )
  )
  (encoder_layer_1): BidirectionalSourceEncoder (
    (forward_encoder): SimpleSourceEncoder (
      (rnn_cell): LSTMCell(256, 128)
    )
    (backward_encoder): SimpleSourceEncoder (
      (rnn_cell): LSTMCell(256, 128)
    )
  )
  (encoder_layer_2): BidirectionalSourceEncoder (
    (forward_encoder): SimpleSourceEncoder (
      (rnn_cell): LSTMCell(256, 128)
    )
    (backward_encoder): SimpleSourceEncoder (
      (rnn_cell): LSTMCell(256, 128)
    )
  )
)

In [25]:
editor.encoder

Encoder (
  (token_embedder): TokenEmbedder (
    (_embedding): TrainFlagEmbedding (
      (_embedding): Embedding(10003, 300)
    )
  )
  (source_encoder): MultiLayerSourceEncoder (
    (encoder_layer_0): BidirectionalSourceEncoder (
      (forward_encoder): SimpleSourceEncoder (
        (rnn_cell): LSTMCell(300, 128)
      )
      (backward_encoder): SimpleSourceEncoder (
        (rnn_cell): LSTMCell(300, 128)
      )
    )
    (encoder_layer_1): BidirectionalSourceEncoder (
      (forward_encoder): SimpleSourceEncoder (
        (rnn_cell): LSTMCell(256, 128)
      )
      (backward_encoder): SimpleSourceEncoder (
        (rnn_cell): LSTMCell(256, 128)
      )
    )
    (encoder_layer_2): BidirectionalSourceEncoder (
      (forward_encoder): SimpleSourceEncoder (
        (rnn_cell): LSTMCell(256, 128)
      )
      (backward_encoder): SimpleSourceEncoder (
        (rnn_cell): LSTMCell(256, 128)
      )
    )
  )
  (edit_encoder): EditEncoder (
    (linear): Linear (256 -> 256)
    (l

In [26]:
input_sentance = "A spokesman for the organization unveiled the plan last week."
insert_words = ""
delete_words = ""
target_sentance = ""


source_words  = word_tokenize(input_sentance)
whitelist     = sorted(word_tokenize(insert_words))
blacklist     = sorted(word_tokenize(delete_words)) 
target_words  = word_tokenize(target_sentance)



In [27]:
ex = EditExample(source_words, [], whitelist, [], blacklist, target_words)
ex

SOURCE: A spokesman for the organization unveiled the plan last week .

In [28]:
import numpy as np

In [29]:
encoder_input = editor.preprocess([ex]).encoder_input
encoder_input

EncoderInput(source_words=SequenceBatch(values=Variable containing:
    6   144    19     4  1670  4175     4   668   249  1275     5
[torch.cuda.LongTensor of size 1x11 (GPU 0)]
, mask=Variable containing:
    1     1     1     1     1     1     1     1     1     1     1
[torch.cuda.FloatTensor of size 1x11 (GPU 0)]
), insert_words=SequenceBatch(values=Variable containing:
 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, mask=Variable containing:
 0
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
), insert_exact_words=SequenceBatch(values=Variable containing:
 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, mask=Variable containing:
 0
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
), delete_words=SequenceBatch(values=Variable containing:
 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, mask=Variable containing:
 0
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
), delete_exact_words=SequenceBatch(values=Variable containing:
 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, mask=Variable conta

In [30]:

sorce_word_embeds = editor.encoder.token_embedder.embed_seq_batch(encoder_input.source_words)
encoder_output = editor.encoder.source_encoder(sorce_word_embeds.split())


In [31]:
encoder_output.combined_states

[SequenceBatchElement(values=Variable containing:

Columns 0 to 9 
-0.0208 -0.2378  0.0203  0.0495  0.1647 -0.0122  0.0347  0.0106 -0.0333 -0.0115

Columns 10 to 19 
-0.0065  0.0470  0.0719  0.0084 -0.0813  0.0752 -0.0061 -0.0960  0.0208  0.0340

Columns 20 to 29 
-0.0126 -0.0423 -0.0212  0.0611  0.0704 -0.0486 -0.1265 -0.0086 -0.1155  0.0682

Columns 30 to 39 
-0.1412  0.0012  0.0128 -0.0038 -0.1056  0.0104 -0.0115  0.0703  0.0227 -0.0354

Columns 40 to 49 
-0.2271  0.0526  0.0120  0.1171 -0.0002  0.0230  0.0468 -0.0175 -0.0438 -0.0334

Columns 50 to 59 
-0.1500 -0.0342  0.0324 -0.0440 -0.0433  0.0165 -0.0262  0.0921  0.0031 -0.0289

Columns 60 to 69 
-0.0623  0.1534 -0.0324  0.0227 -0.0064 -0.0810  0.0515 -0.0051  0.0085 -0.0224

Columns 70 to 79 
 0.0078 -0.0329  0.0033 -0.0021  0.0038  0.0458  0.0107  0.1130  0.0171  0.0387

Columns 80 to 89 
 0.0432 -0.0309  0.0269  0.0344  0.0014  0.0359 -0.0458 -0.0285  0.0049  0.0627

Columns 90 to 99 
 0.0093  0.0422 -0.0124  0.0206  0.1100  0

In [32]:
for state in encoder_output.combined_states:
    print(state.values)

Variable containing:

Columns 0 to 9 
-0.0208 -0.2378  0.0203  0.0495  0.1647 -0.0122  0.0347  0.0106 -0.0333 -0.0115

Columns 10 to 19 
-0.0065  0.0470  0.0719  0.0084 -0.0813  0.0752 -0.0061 -0.0960  0.0208  0.0340

Columns 20 to 29 
-0.0126 -0.0423 -0.0212  0.0611  0.0704 -0.0486 -0.1265 -0.0086 -0.1155  0.0682

Columns 30 to 39 
-0.1412  0.0012  0.0128 -0.0038 -0.1056  0.0104 -0.0115  0.0703  0.0227 -0.0354

Columns 40 to 49 
-0.2271  0.0526  0.0120  0.1171 -0.0002  0.0230  0.0468 -0.0175 -0.0438 -0.0334

Columns 50 to 59 
-0.1500 -0.0342  0.0324 -0.0440 -0.0433  0.0165 -0.0262  0.0921  0.0031 -0.0289

Columns 60 to 69 
-0.0623  0.1534 -0.0324  0.0227 -0.0064 -0.0810  0.0515 -0.0051  0.0085 -0.0224

Columns 70 to 79 
 0.0078 -0.0329  0.0033 -0.0021  0.0038  0.0458  0.0107  0.1130  0.0171  0.0387

Columns 80 to 89 
 0.0432 -0.0309  0.0269  0.0344  0.0014  0.0359 -0.0458 -0.0285  0.0049  0.0627

Columns 90 to 99 
 0.0093  0.0422 -0.0124  0.0206  0.1100  0.1357 -0.0058  0.0734  0.0479